In [ ]:
# -*- coding: utf-8 -*-

# =============================================================================
# Imports
# =============================================================================
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sgn
plt.style.use(["science", "notebook"])
%matplotlib tk

# =============================================================================
# Physical parameters of the simulation 
# =============================================================================

Ngrid = 111 #Number of spatial grid points
x, dx = np.linspace(0.01, 1, Ngrid, retstep = True) #Spatial coordinates and spacing 
nu = 1 #Viscosity coefficient 
u = -4.5*nu/x #Velocity from the analytical expression of equation (1) of the problem set 
u[0] = -abs(u[1]) #outflow boundary conditions
u[-1] = abs(u[-2]) #outflow boundary conditions
Nsteps = 1200 #Number of temporal iteration
dt = abs(dx/(max(u)*50)) #Temporal time step to have stability
beta = 3*nu*dt/dx**2 #Constant from the numerical methods lecture

#Initial condition of the surface density of the accretion disk as a sharp gaussian centered on the peak 
sigma = sgn.gaussian(Ngrid, std = 2) 

#Arrays for the advection and diffusion operators that will be used to update the sigma array at each temporal iteration 
advection = np.copy(sigma)
diffusion = np.copy(sigma)

# =============================================================================
# Numerical Solver and animation
# =============================================================================

plt.ion()
fig, ax = plt.subplots(1,1)

#Plot of the initial condition
ax.plot(x,sigma, '--', label = 'Initial condition')
ax.set_xlim(0,1)
ax.set_ylim(0,1.05)
ax.set_xlabel("x", weight = 'bold', fontsize = 15)
ax.set_ylabel("$\\mathbf{\\Sigma}$ [A.U.]", fontsize = 15, weight = 'bold')
ax.set_title("Temporal Evolution of the Accretion Disk's Surface Density", fontsize = 14, weight = 'bold')

#Plot to be updated during the animation
pl, = ax.plot(x,sigma, 'r', label = 'Evolution')
ax.legend(frameon = True, edgecolor = 'black', fontsize = 15)

#Draw canvas
fig.canvas.draw()

#Temporal evolution
for i in range(Nsteps):
    
    #Advection operator (equation from the numerical methods lecture notes) 
    advection[1:-1] = 0.5*(advection[2:] + advection[:-2] - (u[1:-1]*dt/dx)*(advection[2:] - advection[:-2]))
    
    #Diffusion operator (equation from the numerical methods lecture notes) 
    A = np.eye(Ngrid)*(1. + 2*beta) + np.eye(Ngrid, k=1)* -beta + np.eye(Ngrid, k=-1)*-beta
    diffusion = np.linalg.solve(A,diffusion)
    
    #Update the surface density from the contribution of the advection term and the diffusion term
    sigma[1:-1] = (advection[1:-1] + diffusion[1:-1])
    
    #Outflow boundary condition for the surface density
    sigma[0] = sigma[1]
    sigma[-1] = sigma[-2]
        
    #Update data for the animation 
    pl.set_ydata(sigma)
    fig.canvas.draw()
    plt.pause(0.00001)
    
